<a href="https://colab.research.google.com/github/ananyab1909/Restaurant-Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Hugging Face Transformers and PyTorch
!pip install -q transformers torch scikit-learn pandas nltk tabulate

In [ ]:
# Standard libraries
import re
import uuid
from typing import Dict, Any, Tuple, List

# Data handling
import pandas as pd

# NLP tools
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Hugging Face zero-shot classification
from transformers import pipeline


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install -q transformers torch

In [ ]:
from transformers import pipeline

# Zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# --- Imports ---
import re, uuid
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# --- NLTK setup ---
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenize_text(text):
    try:
        return nltk.word_tokenize(text)
    except:
        return re.findall(r"\b\w+\b", text)

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    tokens = tokenize_text(text)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words and len(t) > 1]
    return " ".join(tokens)

# --- FAQs ---
FAQS = [
    {'id':1,'questions':["What are your opening hours?","When do you close?","What time do you open and close?","Till what time is the restaurant open?"],'answer':"We are open from 10 AM to 11 PM every day."},
    {'id':2,'questions':["Do you have any discounts?","Are there any offers available?","What promotions do you have?","Discounts for parties, festivals, weekends, or students?"],'answer':"We offer 15% off for full bookings (party), 20% during festivals, 10% on Saturday nights (weekend), and 15% for students."},
    {'id':3,'questions':["Can I see the menu?","What food do you serve?","Show me the dishes available","Menu details please"],'answer':"Our menu includes appetizers, main courses, desserts, and beverages."},
    {'id':4,'questions':["What is the cancellation policy?","Can I cancel my booking?","Do you have refunds for cancellations?"],'answer':"You can cancel your booking up to 24 hours before the reservation. Refunds will be processed according to the payment method."},
    {'id':5,'questions':["Do you offer delivery?","What are your delivery timings?","Can I get food delivered?"],'answer':"We deliver until 11 PM every day. Please provide your address for delivery."},
    {'id':6,'questions':["What payment methods do you accept?","Do you accept cards or cash?","Payment options?"],'answer':"We accept cash, debit/credit cards, and UPI payments."},
    {'id':7,'questions':["What is the ambience like?","Tell me about seating and environment","Is it suitable for families or parties?"],'answer':"We have a comfortable ambience suitable for families, parties, and casual dining."},
    {'id':8,'questions':["Can I bring outside food or drinks?","Are outside drinks allowed?","Do you allow external food?"],'answer':"Outside food and drinks are not allowed in the restaurant."},
    {'id':9,'questions':["Do you have facilities for senior citizens?","Wheelchair available?","High chair for toddlers?","Reduced price for toddlers?"],'answer':"We provide wheelchairs for senior citizens and high chairs for toddlers. Reduced prices are available for toddlers."},
    {'id':10,'questions':["Who can enter the bar?","Is under 18 allowed?","Stag entry policy?","Are there age restrictions?"],'answer':"Persons below 18 are not allowed. For stag entry, pay-for-1 policy applies."}
]

# --- TF-IDF setup ---
question_texts, question_map = [], []
for faq in FAQS:
    for q in faq['questions']:
        question_texts.append(preprocess(q))
        question_map.append((faq['id'], q))
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(question_texts)
faq_answer_map = {faq['id']: faq['answer'] for faq in FAQS}

def get_faq_answer(user_text):
    qv = vectorizer.transform([preprocess(user_text)])
    sims = cosine_similarity(qv, X).flatten()
    best_idx = int(sims.argmax())
    fid, _ = question_map[best_idx]
    if sims[best_idx] > 0.45:
        return faq_answer_map[fid], sims[best_idx]
    return None, 0.0

# --- Zero-shot classifier ---
INTENTS = ["booking","takeaway","delivery","ask_discounts","ask_opening_hours","ask_menu","ask_cancellation","ask_bar_rules","ask_ambience","ask_special_requests","smalltalk"]
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def detect_intent_hybrid(user_text):
    # Rule-based first
    text = user_text.lower()
    if any(k in text for k in ["book","reservation","dine in"]): return "booking"
    if any(k in text for k in ["takeaway","pick up","order to go"]): return "takeaway"
    if any(k in text for k in ["delivery","deliver","address"]): return "delivery"
    if any(k in text for k in ["discount","offer","promotion"]): return "ask_discounts"
    if any(k in text for k in ["open","close","hours","time"]): return "ask_opening_hours"
    if any(k in text for k in ["menu","food","dishes"]): return "ask_menu"
    if any(k in text for k in ["cancel","refund"]): return "ask_cancellation"
    if any(k in text for k in ["bar","age","stag","entry"]): return "ask_bar_rules"
    if any(k in text for k in ["ambience","environment","seating"]): return "ask_ambience"
    if any(k in text for k in ["senior","wheelchair","toddler","high chair"]): return "ask_special_requests"

    # Zero-shot fallback
    try:
        result = classifier(user_text, INTENTS)
        if result['scores'][0] >= 0.5:
            return result['labels'][0]
    except:
        pass
    return "smalltalk"

# --- Transaction storage ---
bookings = []
pending_transaction = None
pending_info = {}

def handle_transaction(trans_type, info):
    booking_id = str(uuid.uuid4())[:8]
    bookings.append({"type": trans_type, "info": info, "id": booking_id})
    return f"{trans_type.capitalize()} confirmed! Your reference ID is {booking_id}"

# --- Main chatbot function ---
def respond_to_user(user_text):
    global pending_transaction, pending_info

    # If waiting for details
    if pending_transaction:
        try:
            info = dict(item.strip().split(":") for item in user_text.split(","))
            pending_info.update(info)
            confirmation = handle_transaction(pending_transaction, pending_info)
            pending_transaction = None
            pending_info = {}
            return confirmation
        except:
            return "Invalid format. Please provide details like: name: John, phone: 9998887777"

    # Detect intent first
    intent = detect_intent_hybrid(user_text)

    # Transaction intents first
    if intent in ["booking","takeaway","delivery"]:
        pending_transaction = intent
        pending_info = {}
        return f"Please provide your details for {intent} (name, phone, and address if delivery). Example: 'name: John, phone: 9998887777'"

    # Then FAQ answers
    answer, _ = get_faq_answer(user_text)
    if answer: return answer

    return "I'm here to help with your restaurant queries!"


# --- Interactive loop ---
print("Welcome to the restaurant chatbot! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Bot: Thank you! Have a great day!")
        break
    response = respond_to_user(user_input)
    print(f"Bot: {response}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Device set to use cpu


Welcome to the restaurant chatbot! Type 'exit' to quit.
Bot: Please provide your details for booking (name, phone, and address if delivery). Example: 'name: John, phone: 9998887777'
Bot: Booking confirmed! Your reference ID is af500234
Bot: Please provide your details for delivery (name, phone, and address if delivery). Example: 'name: John, phone: 9998887777'
Bot: Delivery confirmed! Your reference ID is 96d3e7f7
Bot: Please provide your details for takeaway (name, phone, and address if delivery). Example: 'name: John, phone: 9998887777'
Bot: Takeaway confirmed! Your reference ID is f356fb5a
Bot: We are open from 10 AM to 11 PM every day.
Bot: We provide wheelchairs for senior citizens and high chairs for toddlers. Reduced prices are available for toddlers.
Bot: We offer 15% off for full bookings (party), 20% during festivals, 10% on Saturday nights (weekend), and 15% for students.
Bot: Persons below 18 are not allowed. For stag entry, pay-for-1 policy applies.
Bot: I'm here to help w